# Testing which model predicts the best

The first codeblock tests which configuration of hyperparameters works best (by trying all 27 combinations via 2 epoch)
<br><br>
The second codeblock tests the optimal hyperparameter combination via 10 epoch
     

In [ ]:
# MODEL Test 01 

'''
In this model we tested to see what configuration of convolutional_layers/layer_size/dense_layers is best 

We do this by iterating through 3 lists: dense_layers, layer_sizes, conv_layers
We then checked which combination of the above lists resulted in the highest accuracy.

This combination of best hyperparemeters:
[0 dense layers]
[128 layer size]
[1 conv layer]
was then chosen in the second section of this notebook to run with more epochs.

'''

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle
import time
from keras import backend
import datetime

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))


X = X/255.0 #normalizing data

#Hyperparameters of layers
dense_layers = [0,1,2]
layer_sizes = [32,64,128]
conv_layers =[1,2,3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}=-conv-{}-nodes-{}-dense-{}".format(conv_layer,layer_size,dense_layer,int(time.time()))
            tensorboard = TensorBoard(log_dir="C:\\logs\\{}".format(NAME))
            print(NAME)
            
            #layer 1
            model = Sequential()

            model.add(Conv2D(layer_size, (3,3),input_shape = X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            for l in range(conv_layer-1):

            ####################################################
            #adding 3 extra layers

            #layer 2
                model.add(Conv2D(layer_size,(3,3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))
            
            model.add(Flatten())
                              
            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation("relu"))
  

            model.add(Flatten())
            model.add(Dense(64))
            model.add(Activation("relu"))

            model.add(Dense(1))
            model.add(Activation("sigmoid"))

            model.compile(loss      = "binary_crossentropy",
                          optimizer = "adam",
                          metrics   =["accuracy"])

            model.fit(X,y,batch_size=32,epochs =2,validation_split = 0.1,callbacks = [tensorboard]) #pass in 32 photos in each batch a time
            backend.clear_session()


              

# This combination of best hyperparemeters: 
[1 dense layers] <br>
[128 layer size]<br>
[1 conv layer]

In [ ]:
# MODEL Optimal 02

'''
Here we test the model and gain around 60-65% validation accuracy
'''

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle
import time
from keras import backend
import datetime

    
#IMPORTING THE NN FROM PICKLE!
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

#NORMALISING THE DATA
X = X/255.0 #normalizing data



#PICKING THE BEST BEST HYPERPARAMETERS IN TERMS OF AMNT OF LAYERS BASED ON THE PREVIOUS MODEL TEST
dense_layers = [1]
layer_sizes = [128]
conv_layers =[1]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}=-conv-{}-nodes-{}-dense-{}".format(conv_layer,layer_size,dense_layer,int(time.time()))
            tensorboard = TensorBoard(log_dir="C:\\logs\\{}".format(NAME))
            print(NAME)
            
            #layer 1
            model = Sequential()

            model.add(Conv2D(layer_size, (3,3),input_shape = X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            for l in range(conv_layer-1):
            #layer 2
                model.add(Conv2D(layer_size,(3,3)))            
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))
            model.add(Flatten())
                              
            for l in range(dense_layer):
                model.add(Dense(512))
                model.add(Activation("relu"))

            model.add(Dense(1))
            model.add(Activation("sigmoid"))

            model.compile(loss      = "binary_crossentropy",
                          optimizer = "adam",
                          metrics   =["accuracy"])

            model.fit(X,y,batch_size=32,epochs =10,validation_split = 0.1,callbacks = [tensorboard]) #pass in 32 photos in each batch a time
            
  #
#training_acc    is calculated based on amnt of correct on training data
#validation_loss is calculated based on binary_cross_entropy
#val_accuracy   is calculated based on 90% of training set (validation split = 0.1)



# Results
After 10 epochs we found a 0.65% validation accuracy in predicting if the person on the picture was either male/female<br>
this means that on new unseen data the model would perform significantly better than the ZeroR (randomly guessing) algorithm
